In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/gan

/content/drive/MyDrive/gan


In [ ]:
ls

samples/  save/  Untitled0.ipynb


# Import

In [ ]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout,BatchNormalization,Activation
from matplotlib import pyplot
import os
import pylab
import numpy as np

# define the standalone discriminator model

In [ ]:
def define_discriminator(in_shape=(28,28,1)):
	model = Sequential()
	#14*14*32
	model.add(Conv2D(32,kernel_size=3,strides=2,input_shape=in_shape, padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dropout(0.4))
 
	#7*7*64
	model.add(Conv2D(64,kernel_size=3,strides=2,input_shape=in_shape,padding='same'))
	#model.add(BatchNormalization())
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dropout(0.4))

  #3*3*128
	model.add(Conv2D(128,kernel_size=3,strides=2,input_shape=in_shape, padding='same'))
	#model.add(BatchNormalization())
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dropout(0.4))
	model.add(Flatten())
	model.add(Dense(1,activation='sigmoid'))
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

# define the standalone generator model

In [ ]:
def define_generator(latent_dim):
	model = Sequential()
	model.add(Dense(256*7*7, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((7, 7, 256)))

  #14*14*128
	model.add(Conv2DTranspose(128,kernel_size=3,strides=2,
                              padding='same'))
	#model.add(BatchNormalization())
	model.add(LeakyReLU(alpha=0.2))

  # 14*14*64
	model.add(Conv2DTranspose(64,kernel_size=3,strides=1,
                              padding='same'))
	#model.add(BatchNormalization())
	model.add(LeakyReLU(alpha=0.2))

  #28*28*1
	model.add(Conv2DTranspose(1,kernel_size=3,strides=2,
                              padding='same'))
	model.add(Activation('tanh'))
	return model

# define the combined generator and discriminator model, for updating the generator

In [ ]:
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(g_model)
	# add the discriminator
	model.add(d_model)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

# load and prepare mnist training images

In [ ]:
def load_real_samples():
	# load mnist dataset
	(trainX, _), (_, _) = load_data()
	# expand to 3d, e.g. add channels dimension
	X = expand_dims(trainX, axis=-1)
	# convert from unsigned ints to floats
	X = X.astype('float32')
	# scale from [0,255] to [0,1]
	X = X / 255.0
	return X

# select real samples

In [ ]:
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, 1))
	return X, y

# generate points in latent space as input for the generator

In [ ]:
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

# use the generator to generate n fake examples, with class labels

In [ ]:
def generate_fake_samples(g_model, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = g_model.predict(x_input)
	# create 'fake' class labels (0)
	y = zeros((n_samples, 1))
	return X, y

# create and save a plot of generated images (reversed grayscale)

In [ ]:
def save_plot(examples, epoch, n=10):
	# plot images
	for i in range(n * 5):
		# define subplot
		pyplot.subplot(n, 5, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')
	# save plot to file
	sample_dir = 'samples'
	if not os.path.exists(sample_dir):
		os.makedirs(sample_dir) 
	filename = 'generated_plot_e%03d.png' % (epoch+1)
	pyplot.savefig(os.path.join(sample_dir, filename))
	pyplot.close()

# evaluate the discriminator, plot generated images, save generator model

In [ ]:
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, acc_reals, acc_fakes, cnt, n_epochs, n_samples=100):
	# prepare real samples
	X_real, y_real = generate_real_samples(dataset, n_samples)
	# evaluate discriminator on real examples
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
	# evaluate discriminator on fake examples
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print('>Accuracy: [real: %.0f%%] , [fake: %.0f%%]' % (acc_real*100, acc_fake*100))
	# Create a directory if not exists
	save_dir = 'save'
	if not os.path.exists(save_dir):
		os.makedirs(save_dir)
	# Save and plot Statistics
	acc_reals[cnt] = acc_reals[cnt]*(epoch/(epoch+1.)) + acc_real*(1./(epoch+1.))
	acc_fakes[cnt] = acc_fakes[cnt]*(epoch/(epoch+1.)) + acc_fake*(1./(epoch+1.))
	np.save(os.path.join(save_dir, 'acc_fake.npy'), acc_fakes)
	np.save(os.path.join(save_dir, 'acc_real.npy'), acc_reals)
	pyplot.figure()
	pylab.xlim(0, epoch + 1)
	pylab.ylim(0, 1)
	pyplot.plot(range(1, epoch + (n_epochs-(epoch - 1))), acc_fakes, label='acc_fake')
	pyplot.plot(range(1, epoch + (n_epochs-(epoch - 1))), acc_reals, label='acc_real')    
	pyplot.legend()
	pyplot.savefig(os.path.join(save_dir, 'accuracy.pdf'))
	pyplot.close()
	# save plot
	save_plot(x_fake, epoch)
	# save the generator model tile file  
	filename = 'generator_model_%03d.h5' % (epoch + 1)
	g_model.save(os.path.join(save_dir, filename))

# train the generator and discriminator

In [ ]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=50, n_batch=256):
	bat_per_epo = int(dataset.shape[0] / n_batch)
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	d_losses = np.zeros(n_epochs)
	g_losses = np.zeros(n_epochs)
	acc_reals = np.zeros(n_epochs)
	acc_fakes = np.zeros(n_epochs)
	cnt = 0
	for i in range(n_epochs):
		# enumerate batches over the training set
		for j in range(bat_per_epo):
			# get randomly selected 'real' samples
			X_real, y_real = generate_real_samples(dataset, half_batch)
			# generate 'fake' examples
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# create training set for the discriminator
			X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
			# update discriminator model weights
			d_loss, _ = d_model.train_on_batch(X, y)
			# prepare points in latent space as input for the generator
			X_gan = generate_latent_points(latent_dim, n_batch)
			# create inverted labels for the fake samples
			y_gan = ones((n_batch, 1))
			# update the generator via the discriminator's error
			g_loss = gan_model.train_on_batch(X_gan, y_gan)
			# summarize loss on this batch
			print('>epoch: %d --> batch:%d/%d --> [D loss: %.3f] , [G loss: %.3f]' % (i+1, j+1, bat_per_epo, d_loss, g_loss)) 
			d_losses[i] = d_losses[i]*(j/(j+1.)) + d_loss*(1./(j+1.))
			g_losses[i] = g_losses[i]*(j/(j+1.)) + g_loss*(1./(j+1.))
		# evaluate the model performance, sometimes
		if (i+1) % 10 == 0:
			summarize_performance(i, g_model, d_model, dataset, latent_dim, acc_reals, acc_fakes,cnt, n_epochs)
			cnt+=1
		# Create a directory if not exists
		save_dir = 'save'
		if not os.path.exists(save_dir):
			os.makedirs(save_dir)
	 	# Save and plot Statistics
		np.save(os.path.join(save_dir, 'd_losses.npy'), d_losses)
		np.save(os.path.join(save_dir, 'g_losses.npy'), g_losses)
		pyplot.figure()
		pylab.xlim(0, n_epochs + 1)
		pyplot.plot(range(1, n_epochs + 1), d_losses, label='d loss')
		pyplot.plot(range(1, n_epochs + 1), g_losses, label='g loss')    
		pyplot.legend()
		pyplot.savefig(os.path.join(save_dir, 'loss.pdf'))
		pyplot.close()

# call

In [ ]:
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

40960/29515 [=========================================] - 0s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Streaming output truncated to the last 5000 lines.
>epoch: 29 --> batch:155/234 --> [D loss: 0.532] , [G loss: 1.235]
>epoch: 29 --> batch:156/234 --> [D loss: 0.486] , [G loss: 1.106]
>epoch: 29 --> batch:157/234 --> [D loss: 0.498] , [G loss: 1.239]
>epoch: 29 --> batch:158/234 --> [D loss: 0.475] , [G loss: 1.212]
>epoch: 29 --> batch:159/234 --> [D loss: 0.496] , [G loss: 1.162]
>epoch: 29 --> batch:160/234 --> [D loss: 0.493] , [G loss: 1.208]
>epoch: 29 --> batch:161/234 --> [D loss: 0.482] , [G loss: 1.190]
>epoch: 29 --> batch:162/234 --> [D loss: 0.511] , [G loss: 1.208]
>epoch: 29 --> batch:163/234 --> [D loss: 0.484] , [G loss: 1.447]
>epoch: 29 --> batch:164/234 --> [D loss: 0.529] , [G loss: 1.277]
>epoch: 29 --> batch:165/234 --> [D loss: 0.509] , [G loss: 1.256]
>epoch: 29 --> batch:166/234 --> [D loss: 0.473] , [G loss: 1.176]
>epoch: 29 --> batch:167/234 --> [D loss: 0.498] , [G loss: 1.142]
>epoch: 29 --> batch:168/234 --> [D loss: 0.511] , [G loss: 1.284]
>epoch: 29 